In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [108]:
df = pd.read_csv('/content/covid_toy.csv')

In [109]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [110]:
df['city'].value_counts()

,count
city,
Kolkata,32
Bangalore,30
Delhi,22
Mumbai,16


In [111]:
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


Nominal Categorical Variables<br>
Gender and City

Ordinal Categorical Variables<br>
Cough

In [112]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [113]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [114]:
X_train

,age,gender,fever,cough,city
91,38,Male,NaN,Mild,Delhi
40,49,Female,102.0,Mild,Delhi
57,49,Female,99.0,Strong,Bangalore
93,27,Male,100.0,Mild,Kolkata
44,20,Male,102.0,Strong,Delhi
...,...,...,...,...,...
59,6,Female,104.0,Mild,Kolkata
0,60,Male,103.0,Mild,Kolkata
39,50,Female,103.0,Mild,Kolkata
35,82,Female,102.0,Strong,Bangalore


# Without using Column Transformer

In [115]:
# Handling missing values in fever column
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])

X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape, X_test_fever.shape

((80, 1), (20, 1))

In [116]:
# using Ordinal Encoding on Cough column

oe = OrdinalEncoder(categories=[['Mild', 'Strong']]) #smallest value for Mild and Large value for Strong

X_train_cough = oe.fit_transform(X_train[['cough']]) #learn and transform

X_test_cough = oe.transform(X_test[['cough']]) #transform based on X_train

X_train_cough.shape, X_test_cough.shape

((80, 1), (20, 1))

In [117]:
# using OnHotEncoding on gender, city

ohe = OneHotEncoder(drop='first', sparse_output=False)

X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

X_test_gender_city = ohe.transform(X_test[['gender', 'city']])

X_train_gender_city.shape, X_test_gender_city.shape

((80, 4), (20, 4))

In [118]:
# Extracting Age

X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape, X_test_age.shape

((80, 1), (20, 1))

In [126]:
# Concatenating All in One Array
X_transformed_train = np.concatenate((X_train_fever,X_train_cough,X_train_gender_city,X_train_age), axis=1)

X_transformed_test = np.concatenate((X_test_fever,X_test_cough,X_test_gender_city,X_test_age), axis=1)

X_transformed_train.shape, X_transformed_test.shape

((80, 7), (20, 7))

# Using Column **Transformer**


In [120]:
transformer = ColumnTransformer(transformers=[
    ('t1', SimpleImputer(), ['fever']),
    ('t2', OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('t3',OneHotEncoder(sparse_output=False, drop='first'),['gender','city'])
], remainder='passthrough')

In [121]:
X_train_column_transformer = transformer.fit_transform(X_train)

X_test_column_transformer = transformer.transform(X_test)

X_train_column_transformer.shape, X_test_column_transformer.shape

((80, 7), (20, 7))

# Testing if Both Procedures are Equal

Train

In [122]:
if np.array_equal(X_train_column_transformer, X_transformed_train):
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


In [123]:
if (X_train_column_transformer == X_transformed_train).all():
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


Test

In [127]:
if np.array_equal(X_test_column_transformer, X_transformed_test):
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


In [128]:
if (X_test_column_transformer == X_transformed_test).all():
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


# <h1></h1>